In [20]:
# async jupyter notebook execution
import nest_asyncio
nest_asyncio.apply()

import mosaik
import mosaik.util
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

class PVSim:
    def __init__(self):
        self.sid = None
        self.step_size = 60  # 1 hour in minutes
        self.entities = {}
        
    def init(self, sid):
        self.sid = sid
        return {
            'type': 'time-based',
            'models': {
                'PVPlant': {
                    'public': True,
                    'params': ['num'],
                    'attrs': ['p']
                },
                'DispatchableGen': {
                    'public': True,
                    'params': ['num'],
                    'attrs': ['p', 'p_max', 'p_set']
                }
            }
        }
    
    def create(self, num, model, **model_params):
        entities = []
        
        for i in range(num):
            eid = f'{model}_{i}'
            
            if model == 'PVPlant':
                capacity = np.random.uniform(20, 50)  # kW
                self.entities[eid] = {
                    'type': model,
                    'capacity': capacity,
                    'p': 0
                }
            elif model == 'DispatchableGen':
                capacity = np.random.uniform(40, 100)  # kW
                self.entities[eid] = {
                    'type': model,
                    'capacity': capacity,
                    'p': 0,
                    'p_max': capacity,
                    'p_set': 0
                }
            
            entities.append({
                'eid': eid,
                'type': model
            })
            
        return entities
        
    def step(self, time, inputs, max_advance):
        hour = (time // 60) % 24
        
        # Simple solar generation profile
        solar_factor = max(0, np.sin(np.pi * (hour - 6) / 12)) if 6 <= hour <= 18 else 0
        
        for eid, entity in self.entities.items():
            if entity['type'] == 'PVPlant':
                # Add some randomness to solar generation
                noise = np.random.normal(1, 0.1)
                entity['p'] = entity['capacity'] * solar_factor * noise
            elif entity['type'] == 'DispatchableGen':
                # Use the power setpoint for dispatchable generators
                if eid in inputs and 'p_set' in inputs[eid]:
                    entity['p_set'] = next(iter(inputs[eid]['p_set'].values()))
                entity['p'] = min(entity['p_set'], entity['capacity'])
        
        return time + self.step_size
        
    def get_data(self, outputs):
        data = {}
        for eid, attrs in outputs.items():
            data[eid] = {}
            for attr in attrs:
                if attr in ['p', 'p_max', 'p_set']:
                    data[eid][attr] = self.entities[eid][attr]
        return data

# Define custom forecasting simulator
class LoadForecaster:
    def __init__(self, algorithm='arima'):
        self.sid = None
        self.algorithm = algorithm
        self.step_size = 60
        self.data = {}
        self.forecasts = {}
        
    def init(self, sid, algorithm=None):
        if algorithm:
            self.algorithm = algorithm
        self.sid = sid
        return {
            'type': 'time-based',
            'models': {
                'LoadForecaster': {
                    'public': True,
                    'params': ['algorithm'],
                    'attrs': ['p_in', 'forecast']  # Changed to p_in for input
                }
            }
        }
        
    def create(self, num, model, **model_params):
        entities = []
        for i in range(num):
            eid = '{0}_{1}'.format(model, i)
            entities.append({'eid': eid, 'type': model})
        return entities
        
    def step(self, time, inputs, max_advance):
        self.time = time
        
        # Process inputs (actual load measurements)
        for eid, attrs in inputs.items():
            if 'p_in' in attrs:  # Changed from p_out to p_in
                if eid not in self.data:
                    self.data[eid] = []
                for src, value in attrs['p_in'].items():
                    self.data[eid].append(value)
        
        # Generate forecasts based on historical data
        for eid in self.data:
            if len(self.data[eid]) >= 24:  # Need some history for forecasting
                forecast = self._forecast(eid)
                self.forecasts[eid] = forecast
                
        return time + self.step_size
    
    def _forecast(self, eid):
        """Generate load forecast based on selected algorithm"""
        if self.algorithm == 'arima':
            return self._arima_forecast(eid)
        elif self.algorithm == 'moving_average':
            return self._moving_average_forecast(eid)
        elif self.algorithm == 'neural_network':
            return self._neural_network_forecast(eid)
        else:
            # Default simple forecast - repeat yesterday's pattern
            return self._simple_forecast(eid)
    
    def _simple_forecast(self, eid):
        """Simple forecast that repeats the pattern from the previous day"""
        history = self.data[eid]
        if len(history) >= 24:
            # Repeat last 24 hours
            return history[-24:]
        else:
            # Not enough data, repeat what we have
            return history
    
    def _moving_average_forecast(self, eid):
        """Simple moving average forecast"""
        history = self.data[eid]
        window = min(72, len(history))  # Use up to 3 days of history
        
        # Calculate moving average with different weights for recency
        weights = np.linspace(0.5, 1.0, window)
        weights = weights / sum(weights)  # Normalize
        
        forecast = []
        for i in range(self.forecast_horizon):
            if i == 0:
                # First hour forecast is weighted average of history
                value = sum(np.array(history[-window:]) * weights)
            else:
                # Subsequent hours use previous forecasts
                data_to_use = history[-window+i:] + forecast
                data_to_use = data_to_use[-window:]
                value = sum(np.array(data_to_use) * weights)
            forecast.append(value)
            
        return forecast
    
    def _arima_forecast(self, eid):
        """Placeholder for ARIMA forecasting"""
        # In a real implementation, you'd use statsmodels ARIMA
        # This is a simplified version that mimics ARIMA behavior
        history = np.array(self.data[eid])
        forecast = []
        
        # Simple AR(1) model with some randomness
        if len(history) > 0:
            last_value = history[-1]
            mean = np.mean(history)
            for _ in range(self.forecast_horizon):
                # AR(1) formula: x_t = c + φx_{t-1} + ε_t
                phi = 0.8  # Autoregression parameter
                c = mean * (1 - phi)  # Constant
                epsilon = np.random.normal(0, 0.1 * np.std(history) if len(history) > 1 else 0.1)
                next_value = c + phi * last_value + epsilon
                forecast.append(next_value)
                last_value = next_value
        
        return forecast
    
    def _neural_network_forecast(self, eid):
        """Placeholder for neural network forecasting"""
        # This would use a neural network library in practice
        # Here we just implement a simple approximation
        history = np.array(self.data[eid])
        if len(history) < 24 * 7:  # Need at least a week of data
            return self._simple_forecast(eid)
            
        # Simulate daily and weekly patterns
        daily_pattern = np.array([history[i::24].mean() for i in range(24)])
        day_of_week = (self.time // (24 * 60)) % 7
        
        forecast = []
        for i in range(self.forecast_horizon):
            hour = (self.time // 60 + i) % 24
            dow = (day_of_week + (hour + i) // 24) % 7
            
            # Base on daily pattern
            base = daily_pattern[hour]
            
            # Add weekday/weekend effect
            weekday_factor = 0.8 if dow < 5 else 1.2
            
            # Add trend and noise
            trend = 0.001 * i  # Small upward trend
            noise = np.random.normal(0, 0.05 * base)
            
            value = base * weekday_factor + trend + noise
            forecast.append(value)
            
        return forecast
    
    def get_data(self, outputs):
        data = {}
        for eid, attrs in outputs.items():
            data[eid] = {}
            for attr in attrs:
                if attr == 'forecast':
                    data[eid][attr] = self.forecasts.get(eid, [])
                elif attr == 'p_in':
                    if eid in self.data and self.data[eid]:
                        data[eid][attr] = self.data[eid][-1]
                    else:
                        data[eid][attr] = 0
        return data

# Example of custom load simulator implementation
# This would typically be in a separate file: custom_loadsim.py
class LoadSim:
    def __init__(self):
        self.sid = None
        self.step_size = 60  # 1 hour in minutes
        self.entities = {}
        
    def init(self, sid):
        self.sid = sid
        return {
            'type': 'time-based',
            'models': {
                'Houses': {
                    'public': True,
                    'params': ['num'],
                    'attrs': ['p_out']  # This is the output attribute we'll use
                },
                'Commercial': {
                    'public': True,
                    'params': ['num'],
                    'attrs': ['p_out']
                },
                'Industrial': {
                    'public': True,
                    'params': ['num'],
                    'attrs': ['p_out']
                }
            }
        }

    def create(self, num, model, **model_params):
        entities = []
        
        for i in range(num):
            eid = f'{model}_{i}'
            
            if model == 'Houses':
                base_load = np.random.uniform(1, 3)  # kW
            elif model == 'Commercial':
                base_load = np.random.uniform(10, 30)  # kW
            elif model == 'Industrial':
                base_load = np.random.uniform(50, 100)  # kW
            else:
                raise ValueError(f"Unknown load type: {model}")
            
            self.entities[eid] = {
                'type': model,
                'base_load': base_load,
                'p_out': 0
            }
            
            entities.append({
                'eid': eid,
                'type': model
            })
            
        return entities
        
    def step(self, time, inputs, max_advance):
        hour = (time // 60) % 24
        day = (time // (24 * 60)) % 7
        
        # Load patterns for different times of day
        for eid, entity in self.entities.items():
            base_load = entity['base_load']
            
            # Time-of-day factors
            if hour >= 23 or hour < 6:  # Night
                tod_factor = 0.3
            elif 6 <= hour < 9:  # Morning peak
                tod_factor = 1.0
            elif 9 <= hour < 17:  # Working hours
                tod_factor = 0.7
            elif 17 <= hour < 23:  # Evening peak
                tod_factor = 0.9
            
            # Weekend reduction
            day_factor = 0.7 if day >= 5 else 1.0  # Weekend vs weekday
            
            # Add some random variation
            noise = np.random.normal(1, 0.1)
            
            # Calculate final load
            load = base_load * tod_factor * day_factor * noise
            
            # Update entity
            entity['p_out'] = max(0, load)  # Ensure non-negative
            
        return time + self.step_size
        
    def get_data(self, outputs):
        data = {}
        for eid, attrs in outputs.items():
            data[eid] = {}
            for attr in attrs:
                if attr == 'p_out':
                    data[eid][attr] = self.entities[eid]['p_out']
        return data
        
# Example of custom storage simulator implementation
# This would typically be in a separate file: custom_storagesim.py
class StorageSim:
    def __init__(self):
        self.sid = None
        self.step_size = 60  # 1 hour in minutes
        self.entities = {}
        self.time = 0
        
    def init(self, sid):
        self.sid = sid
        return {
            'type': 'time-based',
            'models': {
                'Battery': {
                    'public': True,
                    'params': ['capacity', 'max_power'],
                    'attrs': ['soc', 'power_setpoint']
                }
            }
        }
    
    def create(self, num, model, **model_params):
        entities = []
        
        for i in range(num):
            eid = '{0}_{1}'.format(model, i)
            
            if model == 'Battery':
                # Battery storage
                capacity = np.random.uniform(50, 100)  # kWh
                max_power = np.random.uniform(10, 20)  # kW
                efficiency = np.random.uniform(0.92, 0.98)  # Efficiency
                initial_soc = np.random.uniform(0.3, 0.7)  # Initial state of charge
            else:
                raise ValueError(f"Unknown storage type: {model}")
            
            self.entities[eid] = {
                'type': model,
                'capacity': capacity,
                'max_power': max_power,
                'efficiency': efficiency,
                'soc': initial_soc,
                'energy': capacity * initial_soc,
                'power_setpoint': 0
            }
            
            entities.append({'eid': eid, 'type': model})
            
        return entities
    
    def step(self, time, inputs, max_advance):
        self.time = time
        dt = self.step_size / 60  # Convert to hours
        
        # Process control inputs
        for eid, attrs in inputs.items():
            for attr, values in attrs.items():
                for src, value in values.items():
                    if attr == 'power_setpoint' and eid in self.entities:
                        if isinstance(value, dict) and 'power_setpoint' in value:
                            self.entities[eid]['power_setpoint'] = value['power_setpoint']
                        else:
                            self.entities[eid]['power_setpoint'] = value
        
        # Update all storage entities
        for eid, entity in self.entities.items():
            # Get power setpoint (positive for charging, negative for discharging)
            power = entity['power_setpoint']
            
            # Limit power to max power
            power = max(-entity['max_power'], min(power, entity['max_power']))
            
            # Calculate energy change
            if power >= 0:  # Charging
                energy_change = power * entity['efficiency'] * dt
            else:  # Discharging
                energy_change = power * dt / entity['efficiency']
            
            # Update energy and SOC
            new_energy = entity['energy'] + energy_change
            
            # Constrain energy within capacity limits
            new_energy = max(0, min(new_energy, entity['capacity']))
            
            # Update entity state
            entity['energy'] = new_energy
            entity['soc'] = new_energy / entity['capacity']
            
        return time + self.step_size
    
    def get_data(self, outputs):
        data = {}
        for eid, attrs in outputs.items():
            data[eid] = {}
            for attr in attrs:
                if attr == 'soc':
                    data[eid][attr] = self.entities[eid]['soc']
                elif attr == 'energy':
                    data[eid][attr] = self.entities[eid]['energy']
                elif attr == 'power':
                    data[eid][attr] = self.entities[eid]['power_setpoint']
        return data


# Example of custom wind simulator implementation
# This would typically be in a separate file: custom_windsim.py
class WindSim:
    def __init__(self):
        self.sid = None
        self.step_size = 60  # 1 hour in minutes
        self.entities = {}
        self.time = 0
        self.wind_data = self._generate_synthetic_wind_data(24 * 30)  # 30 days of hourly data
        
    def init(self, sid):
        self.sid = sid
        return {
            'type': 'time-based',
            'models': {
                'WindPlant': {
                    'public': True,
                    'params': ['capacity'],
                    'attrs': ['p']
                }
            }
        }
    
    def create(self, num, model, **model_params):
        entities = []
        
        for i in range(num):
            eid = '{0}_{1}'.format(model, i)
            
            if model == 'WindPlant':
                # Wind power plant
                capacity = np.random.uniform(20, 50)  # kW
                # Add some variation between plants
                phase_shift = np.random.randint(0, 24)  # Different wind patterns
                scale_factor = np.random.uniform(0.8, 1.2)  # Scaling factor
            else:
                raise ValueError(f"Unknown generator type: {model}")
            
            self.entities[eid] = {
                'type': model,
                'capacity': capacity,
                'phase_shift': phase_shift,
                'scale_factor': scale_factor,
                'p': 0  # Current power output
            }
            
            entities.append({'eid': eid, 'type': model})
            
        return entities
    
    def _generate_synthetic_wind_data(self, length):
        """Generate synthetic wind data with realistic patterns"""
        # Base wind speed with daily and seasonal patterns
        time = np.arange(length)
        
        # Daily pattern
        daily_pattern = 2 * np.sin(2 * np.pi * time / 24) 
        
        # Longer term pattern (weather systems)
        weather_pattern = 3 * np.sin(2 * np.pi * time / (24 * 5))  # 5-day cycle
        
        # Randomness
        noise = np.random.normal(0, 1, length)
        
        # Combine patterns
        wind_speed = 5 + daily_pattern + weather_pattern + noise
        wind_speed = np.maximum(0, wind_speed)  # No negative wind speed
        
        # Convert wind speed to capacity factor using power curve approximation
        # Power curve: 0 below cut-in, cubic between cut-in and rated, constant at rated until cut-out
        cut_in = 3.5
        rated = 12.0
        cut_out = 25.0
        
        capacity_factor = np.zeros(length)
        
        for i, speed in enumerate(wind_speed):
            if speed < cut_in or speed > cut_out:
                capacity_factor[i] = 0
            elif speed < rated:
                # Cubic relationship between cut-in and rated
                capacity_factor[i] = ((speed - cut_in) / (rated - cut_in))**3
            else:
                capacity_factor[i] = 1.0
        
        return capacity_factor
    
    def step(self, time, inputs, max_advance):
        self.time = time
        hour_index = (time // 60) % len(self.wind_data)
        
        # Update all entities
        for eid, entity in self.entities.items():
            # Get wind availability with phase shift for diversity
            index = (hour_index + entity['phase_shift']) % len(self.wind_data)
            availability = self.wind_data[index] * entity['scale_factor']
            
            # Calculate power output
            entity['p'] = entity['capacity'] * availability
            
        return time + self.step_size
    
    def get_data(self, outputs):
        data = {}
        for eid, attrs in outputs.items():
            data[eid] = {}
            for attr in attrs:
                if attr == 'p':
                    data[eid][attr] = self.entities[eid]['p']
        return data

# Define control policy simulator
class GridController:
    def __init__(self, control_policy='rule_based'):
        self.sid = None
        self.control_policy = control_policy
        self.step_size = 60  # 1 hour in minutes
        self.forecasts = {}
        self.loads = {}
        self.generators = {}
        self.storage = {}
        self.control_actions = {}
        self.time = 0
        
    def init(self, sid, control_policy=None):
        if control_policy:
            self.control_policy = control_policy
        self.sid = sid
        return {
            'type': 'time-based',
            'models': {
                'GridController': {
                    'public': True,
                    'params': ['control_policy'],
                    'attrs': ['control_action', 'energy_balance']
                }
            }
        }
    
    def create(self, num, model, **model_params):
        entities = []
        for i in range(num):
            eid = '{0}_{1}'.format(model, i)
            entities.append({'eid': eid, 'type': model})
        return entities
        
    def step(self, time, inputs, max_advance):
        self.time = time
        
        # Process inputs
        for eid, attrs in inputs.items():
            for attr, values in attrs.items():
                for src, value in values.items():
                    if attr == 'forecast':
                        if eid not in self.forecasts:
                            self.forecasts[eid] = []
                        self.forecasts[eid] = value
                    elif attr == 'current_load':
                        if 'load' not in self.loads:
                            self.loads['load'] = {}
                        self.loads['load'][eid] = value
                    elif attr == 'available_power':
                        if 'generation' not in self.generators:
                            self.generators['generation'] = {}
                        self.generators['generation'][eid] = value
                    elif attr == 'soc':  # State of charge
                        if 'storage' not in self.storage:
                            self.storage['storage'] = {}
                        self.storage['storage'][eid] = value
        
        # Apply control policy
        self._apply_control_policy()
                
        return time + self.step_size
    
    def _apply_control_policy(self):
        """Apply the selected control policy"""
        if self.control_policy == 'rule_based':
            self._rule_based_control()
        elif self.control_policy == 'mpc':
            self._model_predictive_control()
        elif self.control_policy == 'optimization':
            self._optimization_based_control()
        else:
            # Default to rule-based
            self._rule_based_control()
            
    def _rule_based_control(self):
        """Simple rule-based control policy"""
        # Initialize control actions
        self.control_actions = {}
        
        # Get total load and generation
        total_load = sum(self.loads.get('load', {}).values())
        total_generation = sum(self.generators.get('generation', {}).values())
        
        # Calculate deficit/surplus
        energy_balance = total_generation - total_load
        
        # Control storage based on energy balance
        for storage_id, soc in self.storage.get('storage', {}).items():
            if energy_balance < 0:  # Deficit - discharge storage
                if soc > 0.2:  # Don't discharge below 20%
                    discharge_power = min(-energy_balance, 10)  # Max discharge rate
                    self.control_actions[storage_id] = {'power_setpoint': -discharge_power}
                else:
                    self.control_actions[storage_id] = {'power_setpoint': 0}
            else:  # Surplus - charge storage
                if soc < 0.9:  # Don't charge above 90%
                    charge_power = min(energy_balance, 10)  # Max charge rate
                    self.control_actions[storage_id] = {'power_setpoint': charge_power}
                else:
                    self.control_actions[storage_id] = {'power_setpoint': 0}
                    
        # Control dispatchable generators
        for gen_id in self.generators.get('generation', {}):
            if 'dispatchable' in gen_id:
                if energy_balance < 0:  # Need more power
                    self.control_actions[gen_id] = {'power_setpoint': min(50, abs(energy_balance))}
                else:
                    self.control_actions[gen_id] = {'power_setpoint': 0}
    
    def _model_predictive_control(self):
        """Model Predictive Control (MPC) policy"""
        # This would be a more sophisticated control strategy using the forecasts
        # and a predictive model to optimize over a time horizon
        
        # Simplified implementation
        self.control_actions = {}
        
        # Get forecasts for the next few hours
        forecast_horizon = min(6, len(next(iter(self.forecasts.values()), [])))
        forecasted_loads = []
        
        for i in range(forecast_horizon):
            total_load = 0
            for load_id, forecast in self.forecasts.items():
                if i < len(forecast):
                    total_load += forecast[i]
            forecasted_loads.append(total_load)
        
        # Plan storage usage based on forecasted load profile
        peak_hours = False
        if len(forecasted_loads) > 0:
            current_load = forecasted_loads[0] if forecasted_loads else 0
            future_loads = forecasted_loads[1:] if len(forecasted_loads) > 1 else []
            
            # Check if we're heading toward peak hours
            peak_hours = any(load > current_load * 1.2 for load in future_loads)
        
        # Control storage based on forecast
        for storage_id, soc in self.storage.get('storage', {}).items():
            if peak_hours and soc < 0.9:  # Charge before peak
                self.control_actions[storage_id] = {'power_setpoint': 10}  # Max charge
            elif not peak_hours and current_load > np.mean(forecasted_loads) and soc > 0.2:
                # Discharge during higher than average loads
                self.control_actions[storage_id] = {'power_setpoint': -10}  # Max discharge
            else:
                self.control_actions[storage_id] = {'power_setpoint': 0}
                
        # Control dispatchable generators
        for gen_id in self.generators.get('generation', {}):
            if 'dispatchable' in gen_id:
                if current_load > np.mean(forecasted_loads) * 1.1:
                    # High load period, turn on generators
                    self.control_actions[gen_id] = {'power_setpoint': 40}
                else:
                    # Normal or low load, minimal generation
                    self.control_actions[gen_id] = {'power_setpoint': 10}
    
    def _optimization_based_control(self):
        """Optimization-based control policy"""
        # In a real implementation, this would use an optimization solver
        # to minimize cost while meeting constraints
        
        # Simplified implementation
        self._model_predictive_control()  # Fall back to MPC for now
            
    def get_data(self, outputs):
        data = {}
        for eid, attrs in outputs.items():
            data[eid] = {}
            for attr in attrs:
                if attr == 'control_action' and eid in self.control_actions:
                    data[eid][attr] = self.control_actions[eid]
                elif attr == 'energy_balance':
                    total_load = sum(self.loads.get('load', {}).values())
                    total_generation = sum(self.generators.get('generation', {}).values())
                    data[eid][attr] = total_generation - total_load
        return data


# Main simulation function
def run_smart_grid_simulation(
    sim_duration=24*7,
    load_forecast_algorithm='moving_average',
    control_policy='rule_based',
    include_renewables=True,
    include_storage=True
):
    # Create World
    world = mosaik.World(sim_config)
    
    # Start simulators
    pvsim = world.start('PVSim')
    loadsim = world.start('LoadSim')
    forecaster = world.start('LoadForecaster', algorithm=load_forecast_algorithm)
    controller = world.start('GridController', control_policy=control_policy)
    
    if include_storage:
        storagesim = world.start('StorageSim')
    
    if include_renewables:
        windsim = world.start('WindSim')
    
    # Instantiate models
    houses = loadsim.Houses.create(num=10)
    commercial = loadsim.Commercial.create(num=5)
    industrial = loadsim.Industrial.create(num=2)
    
    # Generators
    pv = pvsim.PVPlant.create(num=5)
    dispatchable_gen = pvsim.DispatchableGen.create(num=2)
    
    if include_renewables:
        wind = windsim.WindPlant.create(num=3)
    
    # Storage
    if include_storage:
        batteries = storagesim.Battery.create(num=3)
    
    # Forecaster and controller
    load_forecaster = forecaster.LoadForecaster.create(num=1)[0]  # Get single entity
    grid_controller = controller.GridController.create(num=1)[0]  # Get single entity
    
    # Connect entities
    # Connect loads to forecaster
    for house in houses:
        world.connect(house, load_forecaster, 'p_out', 'p_in')  # Changed attribute names
    
    for comm in commercial:
        world.connect(comm, load_forecaster, 'p_out', 'p_in')  # Changed attribute names
    
    for ind in industrial:
        world.connect(ind, load_forecaster, 'p_out', 'p_in')  # Changed attribute names
    
    # Connect forecaster to controller
    world.connect(load_forecaster, grid_controller, 'forecast')
    world.connect(load_forecaster, grid_controller, 'p_in', 'current_load')  # Updated attribute name
    
    # Connect generators to controller
    for plant in pv:
        world.connect(plant, grid_controller, 'p', 'available_power')
    
    for gen in dispatchable_gen:
        world.connect(gen, grid_controller, 'p_max', 'available_power')
    
    if include_renewables:
        for plant in wind:
            world.connect(plant, grid_controller, 'p', 'available_power')
    
    # Connect storage to controller
    if include_storage:
        for battery in batteries:
            world.connect(battery, grid_controller, 'soc')
            world.connect(grid_controller, battery, 'control_action', 'power_setpoint')
    
    # Connect controller to dispatchable generators
    for gen in dispatchable_gen:
        world.connect(grid_controller, gen, 'control_action', 'p_set')
    
    # Data collection for analysis
    load_data = mosaik.util.collect_dataframe(
        world, houses + commercial + industrial, 'p_out')  # Consistent with attr name
    
    forecast_data = mosaik.util.collect_dataframe(world, [load_forecaster], 'forecast')
    
    generation_data = mosaik.util.collect_dataframe(
        world, pv + dispatchable_gen, 'p')
        
    if include_renewables:
        wind_data = mosaik.util.collect_dataframe(world, wind, 'p')
        generation_data = pd.concat([generation_data, wind_data], axis=1)
    
    if include_storage:
        storage_data = mosaik.util.collect_dataframe(world, batteries, 'soc')
    
    # Run simulation
    print(f"Starting simulation with {load_forecast_algorithm} forecasting and {control_policy} control")
    world.run(until=sim_duration * 60)  # Convert to minutes
    
    # Return collected data for analysis
    results = {
        'load_data': load_data,
        'generation_data': generation_data,
        'forecast_data': forecast_data
    }
    
    if include_storage:
        results['storage_data'] = storage_data
        
    print(f"Simulation completed after {sim_duration} hours")
    return results

# Simulator configurations
# Update the simulator configurations
sim_config = {
    'PVSim': {
        'python': '__main__:PVSim'  # Use our custom PVSim
    },
    'LoadSim': {
        'python': '__main__:LoadSim'
    },
    'StorageSim': {
        'python': '__main__:StorageSim'
    },
    'WindSim': {
        'python': '__main__:WindSim'
    },
    'LoadForecaster': {
        'python': '__main__:LoadForecaster'
    },
    'GridController': {
        'python': '__main__:GridController'
    }
}

# Example of usage
if __name__ == "__main__":
    # Run simulations with different configurations
    
    # Comparing forecasting algorithms
    algorithms = ['simple', 'moving_average', 'arima', 'neural_network']
    results = {}
    
    for algorithm in algorithms:
        results[algorithm] = run_smart_grid_simulation(
            sim_duration=24*3,  # 3 days
            load_forecast_algorithm=algorithm,
            control_policy='rule_based'
        )
    
    # Calculate forecast accuracy
    for algorithm, data in results.items():
        load_actual = data['load_data'].sum(axis=1)
        load_forecast = data['forecast_data']
        
        # Calculate MAPE (Mean Absolute Percentage Error)
        errors = []
        for i in range(len(load_forecast) - 24):  # For each forecast point
            forecast = load_forecast.iloc[i]
            actual = load_actual.iloc[i:i+24]
            # Calculate error for each hour in the forecast horizon
            for h in range(min(24, len(actual))):
                if actual.iloc[h] > 0:  # Avoid division by zero
                    error = abs(forecast[h] - actual.iloc[h]) / actual.iloc[h]
                    errors.append(error)
        
        mape = np.mean(errors) * 100 if errors else 0
        print(f"{algorithm} forecasting MAPE: {mape:.2f}%")
    
    # Comparing control policies
    policies = ['rule_based', 'mpc', 'optimization']
    control_results = {}
    
    for policy in policies:
        control_results[policy] = run_smart_grid_simulation(
            sim_duration=24*3,  # 3 days
            load_forecast_algorithm='moving_average',  # Use same forecasting for fair comparison
            control_policy=policy
        )
    
    # Analyze control performance
    for policy, data in control_results.items():
        load = data['load_data'].sum(axis=1)
        generation = data['generation_data'].sum(axis=1)
        
        # Calculate energy balance
        balance = generation - load
        
        # Calculate metrics
        energy_deficit = sum(b for b in balance if b < 0)
        energy_surplus = sum(b for b in balance if b > 0)
        balance_std = balance.std()  # Stability measure
        
        print(f"{policy} control policy performance:")
        print(f"  Energy deficit: {abs(energy_deficit):.2f} kWh")
        print(f"  Energy surplus: {energy_surplus:.2f} kWh")
        print(f"  Grid stability (std of balance): {balance_std:.2f} kW")
    
    # Plot results
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 1, 1)
    for algorithm, data in results.items():
        load_actual = data['load_data'].sum(axis=1)
        forecast_horizon = 24
        forecast_time = 12  # Use forecast from 12 hours into simulation
        
        forecast = data['forecast_data'].iloc[forecast_time]
        actual = load_actual.iloc[forecast_time:forecast_time+forecast_horizon]
        
        plt.plot(range(len(forecast)), forecast, label=f"{algorithm} forecast")
    
    plt.plot(range(len(actual)), actual, 'k--', label='Actual load')
    plt.title('Load Forecast Comparison')
    plt.xlabel('Hours ahead')
    plt.ylabel('Power (kW)')
    plt.legend()
    
    plt.subplot(2, 1, 2)
    for policy, data in control_results.items():
        load = data['load_data'].sum(axis=1)
        generation = data['generation_data'].sum(axis=1)
        balance = generation - load
        
        plt.plot(range(len(balance)), balance, label=f"{policy} energy balance")
    
    plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    plt.title('Grid Balance Comparison')
    plt.xlabel('Time (hours)')
    plt.ylabel('Power (kW)')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('smart_grid_simulation_results.png')
    plt.show()

# Analysis functions
def analyze_forecast_accuracy(results):
    """Analyze the accuracy of different forecasting algorithms"""
    accuracy_metrics = {}
    
    for algorithm, data in results.items():
        load_actual = data['load_data'].sum(axis=1)
        load_forecast = data['forecast_data']
        
        # Calculate metrics
        mae_values = []  # Mean Absolute Error
        mape_values = []  # Mean Absolute Percentage Error
        rmse_values = []  # Root Mean Square Error
        
        for i in range(len(load_forecast) - 24):  # For each forecast point
            forecast = load_forecast.iloc[i]
            actual = load_actual.iloc[i:i+24]
            
            # Calculate error metrics for each hour in the forecast horizon
            for h in range(min(24, len(actual))):
                # Absolute error
                abs_error = abs(forecast[h] - actual.iloc[h])
                mae_values.append(abs_error)
                
                # Percentage error
                if actual.iloc[h] > 0:  # Avoid division by zero
                    pct_error = abs_error / actual.iloc[h]
                    mape_values.append(pct_error)
                
                # Squared error
                sq_error = (forecast[h] - actual.iloc[h])**2
                rmse_values.append(sq_error)
        
        # Calculate final metrics
        mae = np.mean(mae_values) if mae_values else 0
        mape = np.mean(mape_values) * 100 if mape_values else 0
        rmse = np.sqrt(np.mean(rmse_values)) if rmse_values else 0
        
        accuracy_metrics[algorithm] = {
            'MAE': mae,
            'MAPE': mape,
            'RMSE': rmse
        }
    
    return accuracy_metrics

def analyze_control_performance(results):
    """Analyze the performance of different control policies"""
    performance_metrics = {}
    
    for policy, data in results.items():
        load = data['load_data'].sum(axis=1)
        generation = data['generation_data'].sum(axis=1)
        
        # Calculate energy balance
        balance = generation - load
        
        # Calculate metrics
        energy_deficit = sum(b for b in balance if b < 0)  # Total deficit
        energy_surplus = sum(b for b in balance if b > 0)  # Total surplus
        balance_std = balance.std()  # Stability measure
        
        # Calculate average absolute imbalance
        avg_imbalance = np.mean(np.abs(balance))
        
        # Calculate ramp rates (change in balance)
        ramp_rates = np.abs(np.diff(balance))
        max_ramp = ramp_rates.max()
        avg_ramp = ramp_rates.mean()
        
        performance_metrics[policy] = {
            'Energy Deficit (kWh)': abs(energy_deficit),
            'Energy Surplus (kWh)': energy_surplus,
            'Grid Stability (Std)': balance_std,
            'Avg Imbalance (kW)': avg_imbalance,
            'Max Ramp (kW/h)': max_ramp,
            'Avg Ramp (kW/h)': avg_ramp
        }
    
    return performance_metrics

def plot_detailed_results(forecast_results, control_results):
    """Create detailed plots of simulation results"""
    plt.figure(figsize=(15, 12))
    
    # Plot 1: Load forecast comparison
    plt.subplot(3, 1, 1)
    forecast_time = 12  # Use forecast from 12 hours into simulation
    for algorithm, data in forecast_results.items():
        load_actual = data['load_data'].sum(axis=1)
        forecast_horizon = 24
        
        forecast = data['forecast_data'].iloc[forecast_time]
        actual = load_actual.iloc[forecast_time:forecast_time+forecast_horizon]
        
        plt.plot(range(len(forecast)), forecast, label=f"{algorithm} forecast")
    
    plt.plot(range(len(actual)), actual, 'k--', label='Actual load')
    plt.title('Load Forecast Comparison (24-hour horizon)')
    plt.xlabel('Hours ahead')
    plt.ylabel('Power (kW)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Plot 2: Grid balance comparison
    plt.subplot(3, 1, 2)
    for policy, data in control_results.items():
        load = data['load_data'].sum(axis=1)
        generation = data['generation_data'].sum(axis=1)
        balance = generation - load
        
        plt.plot(range(len(balance)), balance, label=f"{policy} energy balance")
    
    plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    plt.title('Grid Balance Comparison')
    plt.xlabel('Time (hours)')
    plt.ylabel('Power imbalance (kW)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Plot 3: Battery SOC comparison (if available)
    plt.subplot(3, 1, 3)
    for policy, data in control_results.items():
        if 'storage_data' in data:
            storage = data['storage_data']
            avg_soc = storage.mean(axis=1)
            plt.plot(range(len(avg_soc)), avg_soc * 100, label=f"{policy} avg battery SOC")
    
    plt.title('Battery State of Charge Comparison')
    plt.xlabel('Time (hours)')
    plt.ylabel('State of Charge (%)')
    plt.ylim(0, 100)
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('smart_grid_detailed_results.png', dpi=300)
    plt.show()

def create_performance_summary(forecast_metrics, control_metrics):
    """Create summary tables of performance metrics"""
    # Forecast metrics table
    forecast_df = pd.DataFrame.from_dict(forecast_metrics, orient='index')
    forecast_df = forecast_df.round(2)
    
    # Control metrics table
    control_df = pd.DataFrame.from_dict(control_metrics, orient='index')
    control_df = control_df.round(2)
    
    return forecast_df, control_df


# Example of a full experiment
def run_comprehensive_experiment():
    """Run a comprehensive experiment comparing different algorithms and policies"""
    print("Starting comprehensive experiment...")
    
    # Setup experiment configurations
    forecast_algorithms = ['simple', 'moving_average', 'arima', 'neural_network']
    control_policies = ['rule_based', 'mpc', 'optimization']
    
    # Run forecasting experiments
    forecast_results = {}
    print("Testing forecasting algorithms...")
    for algorithm in forecast_algorithms:
        print(f"  Running simulation with {algorithm} forecasting...")
        forecast_results[algorithm] = run_smart_grid_simulation(
            sim_duration=24*7,  # One week
            load_forecast_algorithm=algorithm,
            control_policy='rule_based'  # Keep control policy constant
        )
    
    # Run control policy experiments
    control_results = {}
    print("Testing control policies...")
    for policy in control_policies:
        print(f"  Running simulation with {policy} control policy...")
        control_results[policy] = run_smart_grid_simulation(
            sim_duration=24*7,  # One week
            load_forecast_algorithm='moving_average',  # Keep forecasting algorithm constant
            control_policy=policy
        )
    
    # Analyze results
    print("Analyzing results...")
    forecast_metrics = analyze_forecast_accuracy(forecast_results)
    control_metrics = analyze_control_performance(control_results)
    
    # Create summary tables
    forecast_summary, control_summary = create_performance_summary(forecast_metrics, control_metrics)
    
    print("\nForecasting Algorithm Performance:")
    print(forecast_summary)
    
    print("\nControl Policy Performance:")
    print(control_summary)
    
    # Plot detailed results
    plot_detailed_results(forecast_results, control_results)
    
    return {
        'forecast_results': forecast_results,
        'control_results': control_results,
        'forecast_metrics': forecast_metrics,
        'control_metrics': control_metrics
    }


# Entry point for the smart grid simulator
if __name__ == "__main__":
    # Run the comprehensive experiment
    experiment_results = run_comprehensive_experiment()
    
    # Additional analysis could be performed here
    
    print("Smart grid simulation experiment completed!")


        ____                              _ _
       /    \                            (_) |
  ____/      \  _ __ ___   ___  ___  __ _ _| | __
 /    \      / | '_ ` _ \ / _ \/ __|/ _` | | |/ /
/      \____/  | | | | | | (_) \__ \ (_| | |   <
\      /    \  |_| |_| |_|\___/|___/\__,_|_|_|\_\
 \____/      \____
 /    \      /    \     mosaik: 3.4.0
/      \____/      \       API: 3.0.13
\      /    \      /    Python: 3.10.11
 \____/      \____/         OS: macOS-13.1-arm64-arm-64bit
      \      /            Docs: https://mosaik.readthedocs.io/en/3.4.0/
       \____/     Get in touch: https://github.com/orgs/OFFIS-mosaik/discussions

2025-03-06 00:10:49.909 | INFO     | mosaik.async_scenario:start:361 - Starting "PVSim" as "PVSim-0" ...
2025-03-06 00:10:49.912 | INFO     | mosaik.async_scenario:start:361 - Starting "LoadSim" as "LoadSim-0" ...
2025-03-06 00:10:49.913 | INFO     | mosaik.async_scenario:start:361 - Starting "LoadForecaster" as "LoadForecaster-0" ...
2025-03-06 00:10:49.9

DEPRECATION WARNING: 'time_resolution' is not implemented as argument of PVSim's init function. This might cause an error in future API versions.
DEPRECATION WARNING: 'time_resolution' is not implemented as argument of LoadSim's init function. This might cause an error in future API versions.
DEPRECATION WARNING: 'time_resolution' is not implemented as argument of LoadForecaster's init function. This might cause an error in future API versions.
DEPRECATION WARNING: 'time_resolution' is not implemented as argument of GridController's init function. This might cause an error in future API versions.
DEPRECATION WARNING: 'time_resolution' is not implemented as argument of StorageSim's init function. This might cause an error in future API versions.
DEPRECATION WARNING: 'time_resolution' is not implemented as argument of WindSim's init function. This might cause an error in future API versions.


ScenarioError: While connecting entities, the following errors occurred:
 - The are problems connecting LoadSim-0.Houses_0.p_in to LoadForecaster-0.LoadForecaster_0.p_in:
- the source attribute does not exist
 - The are problems connecting LoadSim-0.Houses_0.p_out to LoadForecaster-0.LoadForecaster_0.p_out:
- the destination attribute does not exist